In [1]:
import numpy as np
import pandas as pd
from tensorly.decomposition import parafac2
import tensorly as tl
from scipy.stats.mstats import gmean
from tensorly.parafac2_tensor import parafac2_to_slice, apply_parafac2_projections
from tensorly.metrics.regression import variance as tl_var
from pybiomart import Server
from tfac.Data_Mod import form_parafac2_tensor, ohsu_var
from tfac.tensor import OHSU_parafac2_decomp, R2Xparafac2, projections_to_factors
tl.set_backend("numpy")

In [2]:
p2slices, treatmentsTime, proteins, chromosomes, IFproteins, histones, geneExpression, RNAGenes, Rproteins = form_parafac2_tensor()
p2slicesB = ohsu_var(p2slices)
parafac2tensor, error = OHSU_parafac2_decomp(p2slicesB, 5)
weights, transform = projections_to_factors(parafac2tensor)

In [3]:
C = pd.DataFrame(transform[1][5])
D = pd.DataFrame(transform[1][1])
#D is chromosomes by components

In [4]:
geneids = RNAGenes

In [5]:
import copy
import gseapy as gp
def ensembl_convert(df, geneids, decimals):
    convtable = pd.DataFrame()
    server = Server(host='http://www.ensembl.org')
    dataset = (server.marts['ENSEMBL_MART_ENSEMBL'].datasets['hsapiens_gene_ensembl'])
    convtable = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name'])
    ourids = copy.deepcopy(geneids)
    if(decimals):
        for a in range(len(ourids)):
            ourids[a] = ourids[a][:ourids[a].index(".") ]
            
    
    newnames = []
    newtens = pd.DataFrame(df)
    newtens["ensembl ids"] = ourids
    droppedids = newtens[~newtens["ensembl ids"].isin(convtable["Gene stable ID"])]
    newtens = newtens[newtens["ensembl ids"].isin(convtable["Gene stable ID"])]
    for ensid in newtens["ensembl ids"]:
        table = convtable[convtable["Gene stable ID"] == ensid]
        table.reset_index(inplace = True)
        newnames.append(table.at[0, "Gene name"])

    newtens["Gene ID"] = newnames
    
    return newtens

def gsea(newtens, component, gene_set):
    prtens = pd.concat((newtens["Gene ID"], newtens[newtens.columns[component]]), axis = 1)
    pre_res = gp.prerank(rnk=prtens, gene_sets=gene_set, processes=16, max_size=5000, permutation_num=1000, weighted_score_type=1, outdir=None, seed=6)
    return pre_res.res2d

In [6]:
newtens = ensembl_convert(C, geneids, False)

In [7]:
bioplanet = []
kegg19 = []
archs4 = []
gobio = []
gocell = []
gomolecule = []
for comp in range(5):
    gobio.append(gsea(newtens, comp, 'GO_Biological_Process_2018'))
    gocell.append(gsea(newtens, comp, 'GO_Cellular_Component_2018'))
    gomolecule.append(gsea(newtens, comp, 'GO_Molecular_Function_2018'))
    kegg19.append(gsea(newtens, comp, 'KEGG_2019_Human'))
    archs4.append(gsea(newtens, comp, 'ARCHS4_Tissues'))
    bioplanet.append(gsea(newtens, comp, 'BioPlanet_2019'))

2020-08-08 11:38:25,682 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2020-08-08 12:08:21,790 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2020-08-08 12:11:29,361 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2020-08-08 12:18:04,514 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2020-08-08 12:21:31,933 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2020-08-08 12:26:47,935 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2020-08-08 12:38:25,830 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2020-08-08 13:07:53,694 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2020-08-08 13:11:17,989 Input gene rankings contains duplicated IDs, Onl

In [12]:
for i in range(5):
    gobio[i].to_csv('gobio'+str(i)+'.csv')
    gocell[i].to_csv('gocell'+str(i)+'.csv')
    gomolecule[i].to_csv('gomolecule'+str(i)+'.csv')
    kegg19[i].to_csv('kegg19'+str(i)+'.csv')
    archs4[i].to_csv('archs'+str(i)+'.csv')
    bioplanet[i].to_csv('bioplanet'+str(i)+'.csv')

In [19]:
for i in range(5):
    print( 'gobio'+str(i)+ ' ' + str(len(gobio[i][gobio[i]["fdr"] <= 0.05])))
for i in range(5):
    print( 'gocell'+str(i)+ ' ' + str(len(gocell[i][gocell[i]["fdr"] <= 0.05])))
for i in range(5):
    print( 'gomolecule'+str(i)+ ' ' + str(len(gomolecule[i][gomolecule[i]["fdr"] <= 0.05])))
for i in range(5):
    print( 'kegg19'+str(i)+ ' ' + str(len(kegg19[i][kegg19[i]["fdr"] <= 0.05])))
for i in range(5):
    print( 'archs'+str(i)+ ' ' + str(len(archs4[i][archs4[i]["fdr"] <= 0.05])))
for i in range(5):
    print( 'bioplanet'+str(i)+ ' ' + str(len(bioplanet[i][bioplanet[i]["fdr"] <= 0.05])))
    

gobio0 4
gobio1 13
gobio2 0
gobio3 17
gobio4 0
gocell0 5
gocell1 9
gocell2 1
gocell3 1
gocell4 0
gomolecule0 0
gomolecule1 5
gomolecule2 3
gomolecule3 1
gomolecule4 2
kegg190 32
kegg191 16
kegg192 1
kegg193 16
kegg194 33
archs0 55
archs1 49
archs2 14
archs3 38
archs4 82
bioplanet0 4
bioplanet1 20
bioplanet2 0
bioplanet3 22
bioplanet4 2


In [21]:
kegg19[1][kegg19[1]["fdr"]<= 0.05]

,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Graft-versus-host disease,-0.619507,-1469.319423,0.000000,0.000000,41,37,IL1A;HLA-C;FAS;HLA-E;HLA-A;IL1B;HLA-B;CD86;HLA...,HLA-DOB;HLA-DQA2;HLA-DQA1;HLA-DOA;HLA-DMA;HLA-...
Allograft rejection,-0.693908,inf,NaN,0.000000,38,35,HLA-C;FAS;HLA-E;HLA-A;HLA-B;CD86;HLA-F;IL12A;C...,HLA-DOB;CD40;HLA-DQA2;HLA-DQA1;HLA-DOA;HLA-DMA...
Autoimmune thyroid disease,-0.691689,inf,NaN,0.000000,53,50,HLA-C;FAS;HLA-E;HLA-A;HLA-B;CD86;HLA-F;TG;CGA;...,HLA-DOB;CD40;HLA-DQA2;HLA-DQA1;HLA-DOA;HLA-DMA...
Intestinal immune network for IgA production,-0.689429,inf,NaN,0.000000,48,45,LTBR;TGFB1;MAP3K14;CCL28;ITGA4;TNFSF13;CD86;CX...,CCL25;HLA-DOB;CD40;TNFSF13B;HLA-DQA2;HLA-DQA1;...
Staphylococcus aureus infection,-0.774372,inf,NaN,0.000000,68,65,KRT10;C3;PTAFR;ITGB2;DEFB1;C5;MASP2;CFD;ITGAM;...,FGG;C3AR1;FCGR2C;CFI;SELPLG;HLA-DOB;C2;ICAM1;F...
Systemic lupus erythematosus,-0.541004,inf,NaN,0.000000,133,52,ACTN4;SNRPB;ACTN1;SSB;SNRPD3;SNRPD1;C3;RO60;TR...,HLA-DOB;CD40;C2;FCGR2A;HLA-DQA2;HLA-DQA1;HLA-D...
Type I diabetes mellitus,-0.511308,inf,NaN,0.000000,43,41,HSPD1;IL1A;HLA-C;FAS;HLA-E;HLA-A;IL1B;CPE;ICA1...,HLA-DOB;HLA-DQA2;HLA-DQA1;HLA-DOA;HLA-DMA;HLA-...
Ribosome,0.922384,1.276452,0.000000,0.000710,153,150,RPL41;RPS2;RPL4;RPS3;RPLP0;RPL23;RPS18;RPS27;R...,RPL41;RPS2;RPL4;RPS3;RPLP0;RPL23;RPS18;RPS27;R...
Oxidative phosphorylation,0.883528,1.220386,0.000000,0.010597,133,120,ATP5F1B;ATP5MC3;COX6B1;COX6A1;UQCRQ;NDUFS5;COX...,ATP5F1B;ATP5MC3;COX6B1;COX6A1;UQCRQ;NDUFS5;COX...
